In [7]:
import datetime
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent = 5)

In [2]:
from numpy import random
import numpy as np
from scipy.stats import binom
from scipy.stats import poisson

In [3]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook, reset_output
from bokeh.layouts import gridplot

In [62]:
import statsmodels.api as sm


In [5]:
output_notebook()

Loading BokehJS ...

In [39]:
def covid_rates(date, num_infected):
    final = {}
    tot = 0
    for i  in range(14):
        p = poisson.pmf(i, 5)
        final[date] = p * num_infected
        tot += p * num_infected

        date -= datetime.timedelta(days = 1)
    if abs(tot - num_infected) > 2:
        assert False
    return final

In [112]:
def get_data():
    df = pd.read_csv('/home/henry/Downloads/results-20200908-233339.csv')
    df['date'] = pd.to_datetime(df['date'])
    dates = df['date'].tolist()
    cases = df['new_cases'].tolist()
    o = {}
    for i in range(len(dates)):
        o[dates[i]] = cases[i]
    return o

def make_poission(dates, cases):
    main_d = {}
    for i in range(len(cases)):
        c = cases[i]
        d = dates[i]
        dd = covid_rates(d, c)
        for key in dd.keys():
            if not main_d.get(key):
                main_d[key] = dd[key]
            else:
                main_d[key] += dd[key]
    return main_d

def get_new_dates(last_date):
    final = []
    for i in range(1,15):
        final.append(last_date + datetime.timedelta(days = i))
    return final

def get_projected_cases(dates, cases):
    x_ = range(len(dates[-14:]))
    X = list(zip(*[x_]))
    xm = sm.add_constant(X)
    model = sm.OLS(cases[-14:], xm) 
    result = model.fit()
    new_dates = dates[-14:] + get_new_dates(last_date = dates[-1])
    y_hat = [x * result.params[1] + result.params[0] for x in range(len(new_dates))]
    return new_dates[-14:], y_hat[-14:]

def make_graph(dates, cases, p_dates, p_cases):
    p = figure(x_axis_type = 'datetime')
    p.line(x=dates, y = cases, legend_label = 'real')
    p.line(x=p_dates, y = p_cases, legend_label = 'poisson', color ='red', line_width = 2)
    return p


def get_data_with_padded():
    data = get_data()
    dates = sorted(data.keys())
    cases = [data[x] for x in dates]
    e_dates, e_cases = get_projected_cases(dates, cases)
    dates += e_dates
    cases += e_cases
    poisson_d = make_poission(dates, cases)
    p_dates = sorted(poisson_d)[0:-14]
    p_cases = [poisson_d[x] for x in p_dates]
    
    p = make_graph(dates = dates, cases = cases, p_dates = p_dates, p_cases = p_cases)
    show(p)
    
get_data_with_padded()



